Atmospheres & Passbands
============================

Setup
-----------------------------

Let's first make sure we have the latest version of PHOEBE 2.3 installed (uncomment this line if running in an online notebook session such as colab).

In [1]:
#!pip install -I "phoebe>=2.3,<2.4"

As always, let's do imports and initialize a logger and a new bundle.

In [2]:
import phoebe
from phoebe import u # units
import numpy as np
import matplotlib.pyplot as plt

logger = phoebe.logger()

b = phoebe.default_binary()

And we'll add a single light curve dataset to expose all the passband-dependent options.

In [3]:
b.add_dataset('lc', times=np.linspace(0,1,101), dataset='lc01')

<ParameterSet: 73 parameters | contexts: constraint, dataset, figure, compute>

Relevant Parameters
-----------------------

An 'atm' parameter exists for each of the components in the system (for each set of compute options) and defines which atmosphere table should be used.

By default, these are set to 'ck2004' (Castelli-Kurucz) but can be set to 'blackbody' as well as 'extern_atmx' and 'extern_planckint' (which are included primarily for direct comparison with PHOEBE legacy).

In [4]:
b['atm']

<ParameterSet: 2 parameters | components: secondary, primary>

In [5]:
b['atm@primary']

<Parameter: atm=ck2004 | keys: description, choices, value, visible_if, copy_for, readonly, advanced, latexfmt>

In [6]:
b['atm@primary'].description

'Atmosphere table'

In [7]:
b['atm@primary'].choices

['phoenix', 'extern_atmx', 'extern_planckint', 'blackbody', 'ck2004']

Note that if you change the value of 'atm' to anything other than 'ck2004', the corresponding 'ld_func' will need to be changed to something other than 'interp' (warnings and errors will be raised to remind you of this).

In [8]:
b['ld_func@primary']

<ParameterSet: EMPTY>

In [9]:
b['atm@primary'] = 'blackbody'

Thu, 17 Sep 2020 10:52 BUNDLE       WARNING ld_mode='interp' not supported by atm='blackbody'.  Either change atm@primary@phoebe01 or ld_mode@primary@lc01.  If not addressed, this warning will continue to be raised and will throw an error at run_compute.


In [10]:
print(b.run_checks())

Run Checks Report: FAIL
ERROR: ld_mode='interp' not supported by atm='blackbody'.  Either change atm@primary@phoebe01 or ld_mode@primary@lc01. (3 affected parameters, affecting run_compute)


In [11]:
b['ld_mode@primary'] = 'manual'
b['ld_func@primary'] = 'logarithmic'

In [12]:
print(b.run_checks())

Run Checks Report: PASS



A 'passband' parameter exists for each passband-dependent-dataset (i.e. not meshes or orbits, but light curves and radial velocities).  This parameter dictates which passband should be used for the computation of all intensities.

In [13]:
b['passband']

<Parameter: passband=Johnson:V | keys: description, choices, value, visible_if, copy_for, readonly, advanced, latexfmt>

The available choices will include both locally installed passbands as well as passbands currently available from the online PHOEBE repository.  If you choose an online-passband, it will be downloaded and installed locally as soon as required by [b.run_compute](../api/phoebe.frontend.bundle.Bundle.run_compute.md).

In [14]:
print(b['passband'].choices)

['Stromgren:y', 'Stromgren:b', 'Tycho:B', 'KELT:R', 'Cousins:I', 'Gaia:RVS', 'Kepler:mean', 'Gaia:G', 'Johnson:V', 'LSST:y3', 'Stromgren:u', 'Hipparcos:Hp', 'LSST:z', 'Johnson:B', 'Johnson:I', 'Tycho:V', 'TESS:T', 'Gaia:RP', 'BRITE:red', 'LSST:i', 'BRITE:blue', 'Cousins:R', 'LSST:r', 'LSST:g', 'LSST:u', 'Stromgren:v', 'Johnson:R', 'Johnson:U', 'Bolometric:900-40000', 'Gaia:BP']


To see your current locally-installed passbands, call [phoebe.list_installed_passbands()](../api/phoebe.list_installed_passbands.md).

In [15]:
print(phoebe.list_installed_passbands())

['Bolometric:900-40000', 'Johnson:R', 'Johnson:V', 'TESS:T']


These installed passbands can be in any of a number of directories, which can be accessed via [phoebe.list_passband_directories()](../api/phoebe.list_passband_directories.md).

The first entry is the global location - this is where passbands can be stored by a server-admin to be available to all PHOEBE-users on that machine.

The second entry is the local location - this is where individual users can store passbands and where PHOEBE will download and install passbands (by default).

In [16]:
print(phoebe.list_passband_directories())

['/home/kyle/.local/lib/python3.8/site-packages/phoebe/atmospheres/tables/passbands/', '/home/kyle/.phoebe/atmospheres/tables/passbands/']


To see the passbands available from the [online repository](http://github.com/phoebe-project/phoebe2-tables), call [phoebe.list_online_passbands()](../api/phoebe.list_online_passbands.md).

In [17]:
print(phoebe.list_online_passbands())

['BRITE:blue', 'BRITE:red', 'Bolometric:900-40000', 'Cousins:I', 'Cousins:R', 'Gaia:BP', 'Gaia:G', 'Gaia:RP', 'Gaia:RVS', 'Hipparcos:Hp', 'Johnson:B', 'Johnson:I', 'Johnson:R', 'Johnson:U', 'Johnson:V', 'KELT:R', 'Kepler:mean', 'LSST:g', 'LSST:i', 'LSST:r', 'LSST:u', 'LSST:y3', 'LSST:z', 'Stromgren:b', 'Stromgren:u', 'Stromgren:v', 'Stromgren:y', 'TESS:T', 'Tycho:B', 'Tycho:V']


Lastly, to manually download and install one of these online passbands, you can do so explicitly via [phoebe.download_passband](../api/phoebe.download_passband.md) or by visiting [tables.phoebe-project.org](http://phoebe-project.org/tables).  See also the tutorial on [updating passbands](./passband_updates.ipynb).

Note that this isn't necessary unless you want to explicitly download passbands before needed by run_compute (perhaps if you're expecting to have unreliable network connection in the future and want to ensure you have all needed passbands).

In [19]:
phoebe.download_passband('Cousins:R')

In [20]:
print(phoebe.list_installed_passbands())

['Bolometric:900-40000', 'Cousins:R', 'Johnson:R', 'Johnson:V', 'TESS:T']
